# 通し課題：クラウドファンディングが始まる前に成功するかを予測する
# Day1特定した説明変数を使って精度を上げる

* DAY1で特定した説明変数

|説明変数|説明|備考|
|:-|:-|:-|
|usd_goal_real|目標額USDドル換算||
|days|期間||
|country|国|ダミー変数|
|category|カテゴリ|ダミー変数|


In [2]:
#ライブラリ読み込み
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix,precision_recall_fscore_support
from sklearn.model_selection import train_test_split # ホールドアウト法に関する関数
from sklearn.model_selection import KFold # 交差検証法に関する関数
from sklearn.preprocessing import StandardScaler

## Day1特定した説明変数を使ってロジステック回帰モデルを作成し汎化性能を確認する

In [3]:
# DAY1で分析時に使用したデータセット読み込み
df_loss = pd.read_csv('DAY1_work_df_loss.csv')
display(df_loss.head())

,Unnamed: 0,usd_goal_real,state_flg,days,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,...,category_Wearables,category_Weaving,category_Web,category_Webcomics,category_Webseries,category_Woodworking,category_Workshops,category_World Music,category_Young Adult,category_Zines
0,0,1533.95,0,58,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,30000.00,0,59,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,45000.00,0,44,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5000.00,0,29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,50000.00,1,34,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### ホールドアウト法

* 全データのうち20%をテストデータとする

In [4]:
X = df_loss.drop(['state_flg','Unnamed: 0'],axis=1).values
y = df_loss["state_flg"].values
# 全データのうち20%をテストデータとする
test_size = 0.2
# ホールドアウト法を実行
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1234)

#### 訓練誤差

In [5]:
# 学習用データでロジステック回帰を学習
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X_train, y_train)

# 学習用データに対する予測を実行
y_est_train = clf.predict(X_train)

# 学習用データに対する対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y_train, y_est_train)))

# 学習用データに対する正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est_train)))

# 学習用データに対するPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est_train)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

対数尤度 = -16.191
正答率 = 53.122%
適合率（Precision） = 73.505%
再現率（Recall） = 33.258%
F1値（F1-score） = 45.796%


#### 汎化誤差

In [6]:
# テスト用データに対する予測を実行
y_est_test = clf.predict(X_test)

# テスト用データに対する対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y_test, y_est_test)))

# テスト用データに対する正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_est_test)))

# テスト用データに対するPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

対数尤度 = -16.290
正答率 = 52.836%
適合率（Precision） = 73.515%
再現率（Recall） = 33.216%
F1値（F1-score） = 45.757%


#### ホールドアウト法 訓練誤差と汎化誤差の比較

|誤差|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|
|訓練誤差|53.122%|73.505%|33.258%|45.796%|
|汎化誤差|52.836%|73.515%|33.216%|45.757%|

* テストデータでの正答率がわずかに下がったが、訓練誤差と汎化誤差に差はあまりなかった
* 過学習ではないと判断する

## 交差検証法
* 交差検証法を用いて汎化誤差を測定する

In [7]:
# グループ数を設定（5分割）
n_split = 5

cross_valid_ac = 0
cross_valid_pr = 0
cross_valid_re = 0
cross_valid_f1 = 0
split_num = 1

y = df_loss["state_flg"].values
X = df_loss.drop(['state_flg','Unnamed: 0'],axis=1).values

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X[train_idx], y[train_idx] #学習用データ
    X_test, y_test = X[test_idx], y[test_idx]     #テスト用データ
    
    # 学習用データを使ってモデルを学習
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
    clf.fit(X_train, y_train)

    # テストデータに対する予測を実行
    y_est_test = clf.predict(X_test)
    
    # テストデータに対する正答率を計算
    ac = 100 * accuracy_score(y_test, y_est_test)
    print("Fold %s"%split_num)
    print('正答率 = {:.3f}%'.format(ac))

    # テスト用データに対するPrecision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
    pr = 100 * precision[0]
    re = 100 * recall[0]
    f1 = 100 * f1_score[0]
    print('適合率（Precision） = {:.3f}%'.format(pr))
    print('再現率（Recall） = {:.3f}%'.format(re))
    print('F1値（F1-score） = {:.3f}%'.format(f1))      
    print()
          
    cross_valid_ac += ac
    cross_valid_pr += pr
    cross_valid_re += re
    cross_valid_f1 += f1
    split_num += 1

# 平均値を最終的な汎化誤差値とする
final_ac = cross_valid_ac / n_split
final_pr = cross_valid_pr / n_split
final_re = cross_valid_re / n_split
final_f1 = cross_valid_f1 / n_split
print("Cross Validation accuracy = %s"%round(final_ac, 3))
print("Cross Validation Precision = %s"%round(final_pr, 3))
print("Cross Validation Recall = %s"%round(final_re, 3))
print("Cross Validation F1-score = %s"%round(final_f1, 3))

Fold 1
正答率 = 53.798%
適合率（Precision） = 72.369%
再現率（Recall） = 36.971%
F1値（F1-score） = 48.940%

Fold 2
正答率 = 51.370%
適合率（Precision） = 76.385%
再現率（Recall） = 26.412%
F1値（F1-score） = 39.252%

Fold 3
正答率 = 53.069%
適合率（Precision） = 73.262%
再現率（Recall） = 33.200%
F1値（F1-score） = 45.693%

Fold 4
正答率 = 53.026%
適合率（Precision） = 73.041%
再現率（Recall） = 33.596%
F1値（F1-score） = 46.023%

Fold 5
正答率 = 55.803%
適合率（Precision） = 71.368%
再現率（Recall） = 43.179%
F1値（F1-score） = 53.805%

Cross Validation accuracy = 53.413
Cross Validation Precision = 73.285
Cross Validation Recall = 34.672
Cross Validation F1-score = 46.743


### DAY1の結果とホールドアウト法汎化誤差と交差検証法の結果を比較

|検証方法|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|
|DAY1|55.376%|71.140%|42.305%|53.058%|
|ホールドアウト法|52.836%|73.515%|33.216%|45.757%|
|交差検証法|53.413%|73.285%|34.672%|46.743%|

* DAY1の結果と比べ、ホールドアウト法、交差検証法を用いると共に正答率が下がったが、過学習と呼ぶほどではないと判断する。


## 精度を上げる
### 標準化を用いて、精度が上がるか検証する

#### ホールドアウト法
* 上記で行ったホールドアウト法に対し、標準化を適用し、結果を比較する

In [8]:
# 学習用データとテストデータに分ける
X = df_loss.drop(['state_flg','Unnamed: 0'],axis=1).values
y = df_loss["state_flg"].values
# 全データのうち20%をテストデータとする
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1234)

In [9]:
# 標準化
stdsc = StandardScaler()
X_train_stand = stdsc.fit_transform(X_train)
X_test_stand = stdsc.transform(X_test)

##### 訓練誤差

In [10]:
# 学習用データでロジステック回帰を学習
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X_train_stand, y_train)

# 学習用データに対する予測を実行
y_est_train = clf.predict(X_train_stand)

# 学習用データに対する対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y_train, y_est_train)))

# 学習用データに対する正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est_train)))

# 学習用データに対するPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est_train)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

対数尤度 = -11.476
正答率 = 66.774%
適合率（Precision） = 69.828%
再現率（Recall） = 77.828%
F1値（F1-score） = 73.611%


##### 汎化誤差

In [11]:
# テスト用データに対する予測を実行
y_est_test = clf.predict(X_test_stand)

# テスト用データに対する対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y_test, y_est_test)))

# テスト用データに対する正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_est_test)))

# テスト用データに対するPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

対数尤度 = -11.539
正答率 = 66.591%
適合率（Precision） = 69.997%
再現率（Recall） = 77.386%
F1値（F1-score） = 73.506%


#### ホールドアウト法標準化 訓練誤差と汎化誤差の比較

|誤差|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|
|訓練誤差|66.774%|69.828%|77.828%|73.611%|
|汎化誤差|66.591%|69.997%|77.386%|73.506%|

* 訓練誤差と汎化誤差に差はあまりなかった
* 過学習ではないと判断する

#### ホールドアウト法の結果とホールドアウト法標準化の汎化誤差を比較

|誤差|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|
|標準化なし|52.836%|73.515%|33.216%|45.757%|
|標準化あり|66.591%|69.997%|77.386%|73.506%|

* 正答率が上がった。
* 標準化の効果があった

### 交差検証法
* 上記で行ったホールドアウト法に対し、標準化を適用し、結果を比較する

In [12]:
# グループ数を設定（5分割）
n_split = 5

cross_valid_ac = 0
cross_valid_pr = 0
cross_valid_re = 0
cross_valid_f1 = 0
split_num = 1

y = df_loss["state_flg"].values
X = df_loss.drop(['state_flg','Unnamed: 0'],axis=1).values

stdsc = StandardScaler()

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X[train_idx], y[train_idx] #学習用データ
    X_test, y_test = X[test_idx], y[test_idx]     #テスト用データ
    
    # 標準化
    X_train_stand = stdsc.fit_transform(X_train)
    X_test_stand = stdsc.transform(X_test)
    
    # 学習用データを使ってモデルを学習
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
    clf.fit(X_train_stand, y_train)

    # テストデータに対する予測を実行
    y_est_test = clf.predict(X_test_stand)
    
    # テストデータに対する正答率を計算
    ac = 100 * accuracy_score(y_test, y_est_test)
    print("Fold %s"%split_num)
    print('正答率 = {:.3f}%'.format(ac))

    # テスト用データに対するPrecision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
    pr = 100 * precision[0]
    re = 100 * recall[0]
    f1 = 100 * f1_score[0]
    print('適合率（Precision） = {:.3f}%'.format(pr))
    print('再現率（Recall） = {:.3f}%'.format(re))
    print('F1値（F1-score） = {:.3f}%'.format(f1))      
    print()
          
    cross_valid_ac += ac
    cross_valid_pr += pr
    cross_valid_re += re
    cross_valid_f1 += f1
    split_num += 1

# 平均値を最終的な汎化誤差値とする
final_ac = cross_valid_ac / n_split
final_pr = cross_valid_pr / n_split
final_re = cross_valid_re / n_split
final_f1 = cross_valid_f1 / n_split
print("Cross Validation accuracy = %s"%round(final_ac, 3))
print("Cross Validation Precision = %s"%round(final_pr, 3))
print("Cross Validation Recall = %s"%round(final_re, 3))
print("Cross Validation F1-score = %s"%round(final_f1, 3))

Fold 1
正答率 = 66.731%
適合率（Precision） = 69.675%
再現率（Recall） = 78.705%
F1値（F1-score） = 73.915%

Fold 2
正答率 = 66.399%
適合率（Precision） = 69.537%
再現率（Recall） = 77.437%
F1値（F1-score） = 73.275%

Fold 3
正答率 = 66.170%
適合率（Precision） = 70.697%
再現率（Recall） = 73.634%
F1値（F1-score） = 72.136%

Fold 4
正答率 = 66.895%
適合率（Precision） = 70.098%
再現率（Recall） = 77.540%
F1値（F1-score） = 73.631%

Fold 5
正答率 = 66.850%
適合率（Precision） = 69.929%
再現率（Recall） = 77.877%
F1値（F1-score） = 73.689%

Cross Validation accuracy = 66.609
Cross Validation Precision = 69.987
Cross Validation Recall = 77.039
Cross Validation F1-score = 73.329


#### 交差検証法の結果と交差検証法標準化の汎化誤差を比較

|誤差|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|
|標準化なし|53.413%|73.285%|34.672%|46.743%|
|標準化あり|66.609%|69.987%|77.039%|73.329%|

* 正答率が上がった。
* 標準化の効果があった

### DAY1の結果とホールドアウト法汎化誤差と交差検証法と標準化の結果を比較

|検証方法|標準化|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|:-|
|DAY1|なし|55.376%|71.140%|42.305%|53.058%|
|ホールドアウト法|なし|52.836%|73.515%|33.216%|45.757%|
|ホールドアウト法|あり|66.591%|69.997%|77.386%|73.506%|
|交差検証法|なし|53.413%|73.285%|34.672%|46.743%|
|交差検証法|あり|66.609%|69.987%|77.039%|73.329%|

* 標準化を行ったことで、精度が上がったことが確認できた。

### SVMでモデルを作成し、精度が上がるか検証する
* 全データで実行すると時間がかかりすぎるため、サンプリングデータで行う。
* サンプリングデータは全体の1割を使用する

In [13]:
# ライブラリインポート
from sklearn.svm import SVC

In [14]:
# ランダムサンプリング 全体の1割で実施
df_loss_sp = df_loss.sample(frac=0.1,random_state=1234)
display(df_loss_sp.head())
print('全数：{}'.format(len(df_loss)))
print('サンプリング：{}'.format(len(df_loss_sp)))

,Unnamed: 0,usd_goal_real,state_flg,days,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,...,category_Wearables,category_Weaving,category_Web,category_Webcomics,category_Webseries,category_Woodworking,category_Workshops,category_World Music,category_Young Adult,category_Zines
166106,189671,500.00,0,40,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
317562,362547,2500.00,1,24,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58733,66851,300.00,0,59,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121183,138283,38793.97,0,29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81342,92736,5000.00,1,40,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


全数：331675
サンプリング：33168


### 交差検証法 標準化あり
* データをサンプリングしたので、ホールドアウト法より交差検証法の結果のほうが信頼できると判断し、交差検証法にて検証
* サポートベクターマシンで交差検証法を実施すると時間がかかった

In [15]:
%%time
# グループ数を設定（5分割）
n_split = 5

cross_valid_ac = 0
cross_valid_pr = 0
cross_valid_re = 0
cross_valid_f1 = 0
split_num = 1

y = df_loss_sp["state_flg"].values
X = df_loss_sp.drop(['state_flg','Unnamed: 0'],axis=1).values

stdsc = StandardScaler()
C = 10
kernel = "rbf"

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X[train_idx], y[train_idx] #学習用データ
    X_test, y_test = X[test_idx], y[test_idx]     #テスト用データ
    
    # 標準化
    X_train_stand = stdsc.fit_transform(X_train)
    X_test_stand = stdsc.transform(X_test)
    print('サンプリング：{}'.format(len(X_train_stand)))
    
    # 学習用データを使ってモデルを学習
    print("start fit")
    
    # clf = SVC(C=C, kernel=kernel, gamma="scale")
    clf = SVC(C=C, kernel=kernel, gamma=1)
    clf.fit(X_train_stand, y_train)
    
    # テストデータに対する予測を実行
    print("start predict")
    y_est_test = clf.predict(X_test_stand)
    
    # テストデータに対する正答率を計算
    ac = 100 * accuracy_score(y_test, y_est_test)
    print("Fold %s"%split_num)
    print('正答率 = {:.3f}%'.format(ac))

    # テスト用データに対するPrecision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
    pr = 100 * precision[0]
    re = 100 * recall[0]
    f1 = 100 * f1_score[0]
    print('適合率（Precision） = {:.3f}%'.format(pr))
    print('再現率（Recall） = {:.3f}%'.format(re))
    print('F1値（F1-score） = {:.3f}%'.format(f1))      
    print()
          
    cross_valid_ac += ac
    cross_valid_pr += pr
    cross_valid_re += re
    cross_valid_f1 += f1
    split_num += 1

# 平均値を最終的な汎化誤差値とする
final_ac = cross_valid_ac / n_split
final_pr = cross_valid_pr / n_split
final_re = cross_valid_re / n_split
final_f1 = cross_valid_f1 / n_split
print("Cross Validation accuracy = %s"%round(final_ac, 3))
print("Cross Validation Precision = %s"%round(final_pr, 3))
print("Cross Validation Recall = %s"%round(final_re, 3))
print("Cross Validation F1-score = %s"%round(final_f1, 3))

サンプリング：26534
start fit
start predict
Fold 1
正答率 = 63.958%
適合率（Precision） = 66.730%
再現率（Recall） = 79.229%
F1値（F1-score） = 72.444%

サンプリング：26534
start fit
start predict
Fold 2
正答率 = 64.471%
適合率（Precision） = 67.157%
再現率（Recall） = 79.369%
F1値（F1-score） = 72.755%

サンプリング：26534
start fit
start predict
Fold 3
正答率 = 64.742%
適合率（Precision） = 68.258%
再現率（Recall） = 76.962%
F1値（F1-score） = 72.349%

サンプリング：26535
start fit
start predict
Fold 4
正答率 = 66.184%
適合率（Precision） = 69.353%
再現率（Recall） = 79.921%
F1値（F1-score） = 74.263%

サンプリング：26535
start fit
start predict
Fold 5
正答率 = 64.511%
適合率（Precision） = 67.025%
再現率（Recall） = 79.147%
F1値（F1-score） = 72.583%

Cross Validation accuracy = 64.773
Cross Validation Precision = 67.705
Cross Validation Recall = 78.925
Cross Validation F1-score = 72.879
Wall time: 29min 24s


### ロジステック回帰とサポートベクターマシンの結果を比較

|手法|検証方法|標準化|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|:-|:-|
|ロジステック回帰|なし(DAY1)|なし|55.376%|71.140%|42.305%|53.058%|
|ロジステック回帰|ホールドアウト法|なし|52.836%|73.515%|33.216%|45.757%|
|ロジステック回帰|ホールドアウト法|あり|66.591%|69.997%|77.386%|73.506%|
|ロジステック回帰|交差検証法|なし|53.413%|73.285%|34.672%|46.743%|
|ロジステック回帰|交差検証法|あり|66.609%|69.987%|77.039%|73.329%|
|SVM|交差検証法|あり|64.773%|67.705%|78.925%|72.879%|

* ロジステック回帰とサポートベクターマシンの結果を比較したが、ロジステック回帰のほうが結果が良かった
* サポートベクターマシンのパラメータをチューニングしてみる

### グリッドサーチを行いSVMの最適なパラメータを見つけて、精度を検証する

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
X = df_loss_sp.drop(['state_flg','Unnamed: 0'],axis=1).values
y = df_loss_sp["state_flg"].values
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1234)

In [18]:
# 標準化
stdsc = StandardScaler()
X_train_stand = stdsc.fit_transform(X_train)
X_test_stand = stdsc.transform(X_test)

In [19]:
%%time
parameters = {'kernel':['rbf'], 'C':[1, 3,5,7,10]}
model = SVC(gamma="scale") # gamma=scaleにすると、gammaが自動で計算される
gcv = GridSearchCV(model, parameters, cv=5,verbose=2)
gcv.fit(X_train_stand, y_train)
print(gcv.best_params_, gcv.best_score_)

# 最適パラメータを用いて識別する
gcv2 = SVC(**gcv.best_params_, gamma="scale") # gamma=scaleにすると、gammaが自動で計算される
gcv2.fit(X_train_stand, y_train)
print("識別精度=",gcv2.score(X_test_stand, y_test))

# 未知のデータを識別する
y_est_test = gcv2.predict(X_test_stand)

# テスト用データに対する正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_est_test)))

# テスト用データに対するPrecision, Recall, F1-scoreを計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] C=1, kernel=rbf .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................................. C=1, kernel=rbf, total= 1.8min
[CV] C=1, kernel=rbf .................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


[CV] .................................. C=1, kernel=rbf, total= 1.8min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 1.9min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 1.9min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 1.9min
[CV] C=3, kernel=rbf .................................................
[CV] .................................. C=3, kernel=rbf, total= 1.9min
[CV] C=3, kernel=rbf .................................................
[CV] .................................. C=3, kernel=rbf, total= 1.9min
[CV] C=3, kernel=rbf .................................................
[CV] .................................. C=3, kernel=rbf, total= 1.9min
[CV] C=3, kernel=rbf .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 69.3min finished


{'C': 1, 'kernel': 'rbf'} 0.651729856033768
識別精度= 0.6410913476032559
正答率 = 64.109%
適合率（Precision） = 66.562%
再現率（Recall） = 80.338%
F1値（F1-score） = 72.804%
Wall time: 1h 15min 10s


* 線形カーネルを使用すると計算が終わらなかったので、結果なし

### グリッドサーチで見つけたパラメータで再度交差検証法にて汎化誤差を確認

In [20]:
%%time
# グループ数を設定（5分割）
n_split = 5

cross_valid_ac = 0
cross_valid_pr = 0
cross_valid_re = 0
cross_valid_f1 = 0
split_num = 1

y = df_loss_sp["state_flg"].values
X = df_loss_sp.drop(['state_flg','Unnamed: 0'],axis=1).values

stdsc = StandardScaler()
C = 1
kernel = "rbf"

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X[train_idx], y[train_idx] #学習用データ
    X_test, y_test = X[test_idx], y[test_idx]     #テスト用データ
    
    # 標準化
    X_train_stand = stdsc.fit_transform(X_train)
    X_test_stand = stdsc.transform(X_test)
    print('サンプリング：{}'.format(len(X_train_stand)))
    
    # 学習用データを使ってモデルを学習
    print("start fit")
    
    clf = SVC(C=C, kernel=kernel, gamma="scale")
    #clf = SVC(C=C, kernel=kernel, gamma=1)
    clf.fit(X_train_stand, y_train)
    
    # テストデータに対する予測を実行
    print("start predict")
    y_est_test = clf.predict(X_test_stand)
    
    # テストデータに対する正答率を計算
    ac = 100 * accuracy_score(y_test, y_est_test)
    print("Fold %s"%split_num)
    print('正答率 = {:.3f}%'.format(ac))

    # テスト用データに対するPrecision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est_test)
    pr = 100 * precision[0]
    re = 100 * recall[0]
    f1 = 100 * f1_score[0]
    print('適合率（Precision） = {:.3f}%'.format(pr))
    print('再現率（Recall） = {:.3f}%'.format(re))
    print('F1値（F1-score） = {:.3f}%'.format(f1))      
    print()
          
    cross_valid_ac += ac
    cross_valid_pr += pr
    cross_valid_re += re
    cross_valid_f1 += f1
    split_num += 1

# 平均値を最終的な汎化誤差値とする
final_ac = cross_valid_ac / n_split
final_pr = cross_valid_pr / n_split
final_re = cross_valid_re / n_split
final_f1 = cross_valid_f1 / n_split
print("Cross Validation accuracy = %s"%round(final_ac, 3))
print("Cross Validation Precision = %s"%round(final_pr, 3))
print("Cross Validation Recall = %s"%round(final_re, 3))
print("Cross Validation F1-score = %s"%round(final_f1, 3))

サンプリング：26534
start fit
start predict
Fold 1
正答率 = 64.109%
適合率（Precision） = 66.562%
再現率（Recall） = 80.338%
F1値（F1-score） = 72.804%

サンプリング：26534
start fit
start predict
Fold 2
正答率 = 64.697%
適合率（Precision） = 66.571%
再現率（Recall） = 82.219%
F1値（F1-score） = 73.573%

サンプリング：26534
start fit
start predict
Fold 3
正答率 = 65.059%
適合率（Precision） = 67.489%
再現率（Recall） = 80.458%
F1値（F1-score） = 73.405%

サンプリング：26535
start fit
start predict
Fold 4
正答率 = 66.380%
適合率（Precision） = 68.905%
再現率（Recall） = 81.872%
F1値（F1-score） = 74.831%

サンプリング：26535
start fit
start predict
Fold 5
正答率 = 65.476%
適合率（Precision） = 67.355%
再現率（Recall） = 81.179%
F1値（F1-score） = 73.624%

Cross Validation accuracy = 65.144
Cross Validation Precision = 67.377
Cross Validation Recall = 81.213
Cross Validation F1-score = 73.647
Wall time: 14min 42s


### グリッドサーチの結果を比較

|手法|検証方法|標準化|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|:-|:-|
|SVM|交差検証法|あり|64.773%|67.705%|78.925%|72.879%|
|SVM|交差検証法(パラメータ調整後)|あり|65.144%|67.377%|81.213%|73.647%|

* パラメータ調整後、正答率が上がった


## まとめ
### ロジステック回帰とサポートベクターマシンの結果を比較

|手法|検証方法|標準化|正答率|適合率|再現率|F1値|
|:-|:-|:-|:-|:-|:-|:-|
|ロジステック回帰|なし(DAY1)|なし|55.376%|71.140%|42.305%|53.058%|
|ロジステック回帰|ホールドアウト法|なし|52.836%|73.515%|33.216%|45.757%|
|ロジステック回帰|ホールドアウト法|あり|66.591%|69.997%|77.386%|73.506%|
|ロジステック回帰|交差検証法|なし|53.413%|73.285%|34.672%|46.743%|
|ロジステック回帰|交差検証法|あり|66.609%|69.987%|77.039%|73.329%|
|SVM|交差検証法|あり|64.773%|67.705%|78.925%|72.879%|
|SVM|交差検証法(パラメータ調整後)|あり|65.144%|67.377%|81.213%|73.647%|

* ホールドアウト法、交差検証法より過学習が起きてないことを確認した。
* 標準化を行うことで精度が向上した。
* ロジステック回帰のほうが結果がよかったが、データの母数に違いがあるのでどちらがいいと判断しづらい
* サポートベクターマシンは実行結果が返ってくるまで時間がかかった。